# ДЗ-2 
## Финтех школа зима 2019
### Усеинов Булат 

# Теоретические вопросы

*1) Какую роль выполняет функция активации в полносвязной нейросети? Также нужно пояснить, почему стакать несколько подряд идущих полносвязных слоев без активации не является хорошей практикой.* **0.5 балла**

*2) Допустим у вас есть MLP ("multi layer perceptron" или полносвязная нейросеть) с 10-ю входными нейронами, за ними 50 нейронов в скрытом слое, на выходе 3 нейрона. Все нейроны снабжены функцией активации ReLU.* **1 балл**
- Какова размерность матрицы $X$ на входе, если обучение проводится батчами. Размер батча $batch\_size$
- Какова размерность матрицы весов скрытого слоя $W_h$ и какова размерность вектора байеса $b_h$ этого скрытого слоя?
- Какова размерность матрицы весов выходного слоя $W_o$ и какова размерность вектора байеса $b_o$ выходного слоя?
- Какова размерность выходной матрицы $Y$?
- Напишите как выглядит функция, которая считает выходную матрицу $Y$ как функцию от $X$, $W_h$, $W_o$, $b_h$, $b_o$.

*3) Как устроено автоматическое дифференцирование в Tensorflow?* **0.5 балла**
- Используются ли символьные вычисления?
- Используется ли численное дифференцирование?
- Сколько проходов по графу вычислений необходимо совершить,
  чтобы посчитать градиенты для каждого параметра в Tensorflow?
  
*4) Что такое "backpropagation" и для чего он нужен. Чем он отличается от алгоритма градиентного спуска?* **0.5 балла**

*5) Расскажите в чем отличия между mini-batch, batch, stochastic gradient descent. Какой из этих алгоритмов самый популярный и почему.* **0.5 балла**

*6) Можно ли инициализировать первоначально все параметры обучаемой модели MLP единицами? А нулями? Поясните ваш ответ. Можно ли инициализировать нулями параметры $b_o$, $b_h$ из задачи №2?* **0.5 балла**

*7) Вывести лосс-функцию кросс-энтропии из правдоподобия и из теории информации.* **0.5 балла**

# Теоретические ответы

1. Функция активации вносит в системы нелинейность, для апроксимации не только линейных функций. Также это требование знаменитой теоремы Колмогорова-Арнольда о представлении непрерывной функции многих переменных суперпозицей функций одной переменной, которая переложена на нейронные сети и требует функции активации и является теоретическим обоснованием применимости нейронных сетей.
2. - размерность матрицы входных данных, в терминах тензоров $[batch\_size, 10]$
   - $W_h: [10, 50]$, $H_1: [batch\_size, 10]$, $b_h: [10, 1]$
   - $W_o: [50, 3]$, $H_2 = [batch\_size, 3]$, $b_o: [50,1]$
3. модуль tf.GradientTape используется для записи операций и для последующего вычисления градиента. Происходит два прохода, данные хранятся на tape. Для вычисления градиента, пленка воспроизводится в обратном порядке. Конкретная запись tf.GradientTape может произвести расчет только одного градиента.
4. backpropagation - алгоритм обратного распространения ошибки, служит для "обучения" весов нейронной сети. Рассчитывает частные производные и использует их в правиле вычисления производной сложной функции. Можно назвать обощение градиентного спуска.
5. - $mini-batch$  - Обучение происходит "кусочками" датасета, модель получает не одну строчку данных а какой-то набор заданного размера, хочется отметить, что зачастую обощающая способность такого метода куда выше других. Да и работает быстрее.
   - $batch$  - Обучение происходит на всем наборе данных из датасета. Работает дольше, да и обобщающая способность на "грязных и шумных" данных не радует.
   - $SGD$  - Обучение происходит на случайно выбранном объекте, плохо сходится из-за достаточно больших флуктуаций в силу случайнсти выбора объекта.
   
   Не сказал бы что можно выделить самый популярный метод, скорее для каждого есть своя задача. Могу выделить $mini-batch$ и $SGD$.
6. -  Единицами не получится, ибо градиенты не будут меняться и останутся одинаковыми.
   -  Нулями не получится, ибо градиенты станут нулевыми и даже утекать будет некуда).
   -  вполне можно, модель будет учиться и с нулевыми Байесовскими векторами.
7. 



# Первые шаги с TensorFlow

[Описание данных](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Подгружаем данные

In [2]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index)
)

california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
13255,-121.9,37.3,42.0,1067.0,256.0,608.0,280.0,3.0,180.8
7982,-118.4,34.0,35.0,684.0,161.0,381.0,159.0,2.8,272.0
16309,-122.5,37.8,52.0,8.0,1.0,13.0,1.0,15.0,500.0
1584,-117.2,32.8,24.0,730.0,196.0,335.0,203.0,3.5,362.5
5350,-118.2,34.1,43.0,1810.0,427.0,742.0,429.0,3.9,350.0


## EDA

Посмотрим на данные

In [3]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


## Первая модель

В этом задании требуется предсказать стоимовть жилья (`median_house_value`).

Мы будем делать это с помощью [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) и [tf estimators](https://www.tensorflow.org/get_started/estimator).

### 1. Определяем target

In [4]:
targets = california_housing_dataframe["median_house_value"]

### 2. Определяем входы в модель

С помощью [Dataset API](https://www.tensorflow.org/programmers_guide/datasets) и [Feature columns](https://www.tensorflow.org/guide/feature_columns) мы с легкостью определяем, как данные будут подаваться в модель.

In [5]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    
    # ключ total_rooms присутствует в features
    features = {key: np.array(value) for key, value in dict(features).items()}                                           
 
    ds = tf.data.Dataset.from_tensor_slices((features, targets))
    # будем выдавать данные батчами в течение num_epochs эпох
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        # делаем шафл только во время обучения
        # сможете ответить, зачем нужен шафл данных?
        ds = ds.shuffle(buffer_size=10000)

    return ds

my_feature = california_housing_dataframe[["total_rooms"]]
# по ключу "total_rooms" feature_column будет вытаскивать из tf.data.Dataset нужные фичи
feature_columns = [tf.feature_column.numeric_column("total_rooms", shape=(1,))]

### 3. Определяем Модель

In [6]:
lr = 0.00001
my_optimizer = tf.train.AdamOptimizer(learning_rate=lr)

# посмотрите, какие аргументы принимает LinearRegressor
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

### 4. Обучаем модель

In [7]:
_ = linear_regressor.train(
    input_fn = lambda: my_input_fn(my_feature, targets),
    steps=100  # 1 step -- это проход по одному батчу
)

### 5. Считаем качество

In [8]:
prediction_input_fn = lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)
# питоновский генератор
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# переводим в numpy
predictions = np.array([item['predictions'][0] for item in predictions])

# считаем метрики
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = np.sqrt(mean_squared_error)

print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 55530.185
Root Mean Squared Error (on training data): 235.648


## Перебор гиперпараметров

In [9]:
def train_model(learning_rate, num_epochs, batch_size, input_feature="total_rooms"):

    my_feature_data = california_housing_dataframe[[input_feature]]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,)),]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [10]:
# нужно было запихнуть массив фичей, сверху видимо что-то не так
def train_model_by_me(learning_rate, num_epochs, batch_size, input_feature):

    my_feature_data = california_housing_dataframe[input_feature]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(x, shape=(1,)) for x in input_feature]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [11]:
def train_model_by_me_with_estimator(learning_rate, num_epochs, batch_size, input_feature):

    my_feature_data = california_housing_dataframe[input_feature]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(x, shape=(1,)) for x in input_feature]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.DNNLinearCombinedRegressor(linear_feature_columns= feature_columns)
    
    

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

## Задание 1 (2 балла)

Попробуйте поиграть с гиперпараметрами и достигнуть $RMSE \leq 180$

Также можно попробовать:
1. Другой оптимайзер ([здесь](https://www.tensorflow.org/api_docs/python/tf/train) можно посмотреть, какие бывают)
2. Комбинацию фичей (**feature_columns** -- это list, в котором несколько tf.feature_column.numeric_column)
3. Поиграть с аргументами модели (посмотрите, какие аргументы есть у LinearRegressor)

p.s.
Попробуйте переписать функцию `train_model`, чтобы было удобней изменять параметры модели.

In [12]:
# пример обучения модели
train_model(
    learning_rate=0.00001,
    num_epochs=1,
    batch_size=1
)

Training model...
Final RMSE (on training data): 206.44


In [13]:
train_model(
    learning_rate=0.0001,
    num_epochs=10,
    batch_size=512
)

Training model...
Final RMSE (on training data): 166.32


## Задание 2 (2 балла)

Попробуйте подать в модель другие фичи и достигнуть $RMSE \leq 170$

In [14]:
california_housing_dataframe.corr()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.0,-0.9,-0.1,0.0,0.1,0.1,0.1,-0.0,-0.0
latitude,-0.9,1.0,0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1
housing_median_age,-0.1,0.0,1.0,-0.4,-0.3,-0.3,-0.3,-0.1,0.1
total_rooms,0.0,-0.0,-0.4,1.0,0.9,0.9,0.9,0.2,0.1
total_bedrooms,0.1,-0.1,-0.3,0.9,1.0,0.9,1.0,-0.0,0.0
population,0.1,-0.1,-0.3,0.9,0.9,1.0,0.9,-0.0,-0.0
households,0.1,-0.1,-0.3,0.9,1.0,0.9,1.0,0.0,0.1
median_income,-0.0,-0.1,-0.1,0.2,-0.0,-0.0,0.0,1.0,0.7
median_house_value,-0.0,-0.1,0.1,0.1,0.0,-0.0,0.1,0.7,1.0


In [17]:
features = ["housing_median_age", "total_rooms", "total_bedrooms", "population","households", "median_income"]

In [18]:
train_model_by_me(
    learning_rate=0.0012,
    num_epochs=30,
    batch_size=512,
    input_feature = features)

Training model...
Final RMSE (on training data): 157.70


## Задание 3 (2 балла)

Вместо `tf.estimator.LinearRegressor` попробуйте применить другую модель и достигнуть $RMSE \leq 160$

In [19]:
train_model_by_me_with_estimator(
    learning_rate=0.0001,
    num_epochs=50,
    batch_size=100,
    input_feature = features)

Training model...
Final RMSE (on training data): 149.25
